In [3]:
from zhipuai import ZhipuAI
import os
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())
def zhipu_embedding(text: str):

    api_key = os.environ['ZHIPUAI_API_KEY']
    client = ZhipuAI(api_key=api_key)
    response = client.embeddings.create(
        model="embedding-2",
        input=text,
    )
    return response


In [4]:
from langchain.document_loaders.pdf import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("/tmp/pycharm_project_369/Stuffs/amazon-bedrock-voice-conversation/amazon-bedrock-voice-conversation/资料.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pages = loader.load()
#pages = pages[1]

In [5]:
import re
pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
pages.page_content=re.sub(pattern, lambda match: match.group(0).replace('\n', ''),pages.page_content)
pages.page_content=pages.page_content.replace(',', '')
pages.page_content=pages.page_content.replace(' ', '')
pages.page_content=pages.page_content.replace('。', '')

In [6]:
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 知识库中单段文本长度
CHUNK_SIZE = 500
# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
split_docs=text_splitter.split_text(pages.page_content[:])


In [11]:
text = split_docs[0:]
response = zhipu_embedding(text=text)
# 定义持久化路径
persist_directory = '../../data_base/vector_db/chroma'

from langchain.vectorstores.chroma import Chroma

vectordb = Chroma.from_documents(
    documents=split_docs[:], # 为了速度，只选择前 20 个切分的 doc 进行生成；使用千帆时因QPS限制，建议选择前 5 个doc
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
vectordb.persist()

AttributeError: 'str' object has no attribute 'page_content'